In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA
import pandas as pd
from textblob import TextBlob

mylist = list()
InputSheet = "Same-Sim-latest-V1-Sample.csv"
MasterSheet = "Same_SIM_Master_List.csv"
resultFrame = pd.DataFrame(columns=('NewCode','NewCodeDesc','SameSimCode', 'CPTDesc','Accuracy','Rank','Comments'),index=range(30000))
rownum = -1
NewCodeNum = -1

def create_tokenizer_score(new_series, train_series, tokenizer):
    """
    return the tf idf score of each possible pairs of documents
    Args:
        new_series (pd.Series): new data (To compare against train data)
        train_series (pd.Series): train data (To fit the tf-idf transformer)
    Returns:
        pd.DataFrame
    """
    
    train_tfidf = tokenizer.fit_transform(train_series)
    
    new_tfidf = tokenizer.transform(new_series)

    X = pd.DataFrame(cosine_similarity(new_tfidf, train_tfidf), columns=train_series.index)
    X['ix_new'] = new_series.index
    score = pd.melt(
        X,
        id_vars='ix_new',
        var_name='ix_train',
        value_name='score'
    )
    return score


def Get_MasterCodeDesc():
    df = pd.read_csv(MasterSheet)
    sMasterCodeDesc = df['CPTLongDesc'].tolist()
    return sMasterCodeDesc;

def GetNewCPT():
    NewCodeFrame = pd.read_csv(InputSheet,encoding ='latin1')
    sNewCode = NewCodeFrame['HCPCS'].tolist()
    return sNewCode;

def GetNewCPTDesc():
    NewCodeFrame = pd.read_csv(InputSheet,encoding ='latin1')
    sNewCodeDesc = NewCodeFrame['LongDesc'].tolist()
    return sNewCodeDesc;

def GetSameSIMData():
    SameSimDf = pd.read_csv(InputSheet,encoding ='latin1')
    sNewCodeDesc = SameSimDf['SimCode'].tolist()
    return sNewCodeDesc;
   
def textblob_tokenizer(str_input):
    blob = TextBlob(str_input.lower())
    tokens = blob.words
    words = [token.stem() for token in tokens]
    return words

def GetMasterCPTCode(Val,NewCode,NewCodeDesc,Accuracy,RankNo,Comment,x):
    df = pd.read_csv(MasterSheet)    
    GetCPTValue = df.iloc[Val,0]
    GetCPTDesc = df.iloc[Val,1]    
    mylist.append(GetCPTValue)
    mylist.append(GetCPTDesc)
    resultFrame.loc[x].NewCode = NewCode
    resultFrame.loc[x].SameSimCode = GetCPTValue
    resultFrame.loc[x].CPTDesc = GetCPTDesc
    resultFrame.loc[x].NewCodeDesc = NewCodeDesc
    resultFrame.loc[x].Accuracy = Accuracy
    resultFrame.loc[x].Rank = RankNo
    resultFrame.loc[x].Comments = Comment
    return mylist;

#train_CPT_Set = pd.Series(GetMasterCPT())
train_set = pd.Series(Get_MasterCodeDesc())
NewCodeDescSet = pd.Series(GetNewCPTDesc())
NewCodeSet = pd.Series(GetNewCPT())
SameSimSet = pd.Series(GetSameSIMData())
for i in NewCodeDescSet:
    NewCodeNum = NewCodeNum+1
    print(NewCodeSet[NewCodeNum])
    test_set = pd.Series(i)
    #print(i)
    #tokenizer = TfidfVectorizer(stop_words = 'english')# initiate here your own tokenizer (TfidfVectorizer, CountVectorizer, with stopwords...)
    tokenizer = TfidfVectorizer()
    score = create_tokenizer_score(train_series=train_set, new_series=test_set, tokenizer=tokenizer)
    #score
    pd.set_option("display.max_rows", 11)
    sortedscore = score.sort_values(by=['score'], ascending=False)
    sortedscore.head(11)
    ranknum = 0
    found = "false"
    for x in range(0,10):
        ranknum = ranknum+1
        df = pd.read_csv(MasterSheet) 
        svalue = sortedscore.iloc[x,1]
        GetCPTValue = df.iloc[svalue,0]
        if (GetCPTValue == SameSimSet[NewCodeNum]):
            found = "true"
            break 
    
    RecNum =0
    for x in range(0,10):
        rownum = rownum+1
        svalue = sortedscore.iloc[x,1]
        Accuracy = sortedscore.iloc[x,2]
        if (RecNum+1==ranknum and found =="true"):
            Comments = "Present Both in MD and Autoproposal"
        else:
            Comments = "Present in Autoproposal"
        if (RecNum ==0):
            result = GetMasterCPTCode(svalue,NewCodeSet[NewCodeNum],i,Accuracy*100,ranknum,Comments,rownum)
        else:
            result = GetMasterCPTCode(svalue,NewCodeSet[NewCodeNum],i,Accuracy*100,"",Comments,rownum)
        RecNum =RecNum +1
 
resultFrame

#export_csv = resultFrame.to_csv ('export_dataframe_nostopwords_V1.csv', index = None, header=True)



0045U
0046U
0047U
0048U
0048U
0049U
0050U
0051U
0051U
0051U
0052U
0053U
0054U
0055U
0056U
0056U
0057U
0058U
0059U
0060U
0060U
0061U
0062U
0062U


,NewCode,NewCodeDesc,SameSimCode,CPTDesc,Accuracy,Rank,Comments
0,0045U,"Oncology (breast ductal carcinoma in situ), mR...",81525,"Oncology (colon), mRNA, gene expression profil...",90.2815,2,Present in Autoproposal
1,0045U,"Oncology (breast ductal carcinoma in situ), mR...",81519,"Oncology (breast), mRNA, gene expression profi...",83.8788,,Present Both in MD and Autoproposal
2,0045U,"Oncology (breast ductal carcinoma in situ), mR...",81541,"Oncology (prostate), mRNA gene expression prof...",76.1083,,Present in Autoproposal
3,0045U,"Oncology (breast ductal carcinoma in situ), mR...",81520,"Oncology (breast), mRNA gene expression profil...",75.2259,,Present in Autoproposal
4,0045U,"Oncology (breast ductal carcinoma in situ), mR...",81521,"Oncology (breast), mRNA, microarray gene expre...",60.8107,,Present in Autoproposal
...,...,...,...,...,...,...,...
29995,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29996,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29998,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
!pip install -U textblob

Requirement not upgraded as not directly required: nltk>=3.1 in c:\programdata\anaconda3\lib\site-packages (from textblob) (3.3)
Requirement not upgraded as not directly required: six in c:\programdata\anaconda3\lib\site-packages (from nltk>=3.1->textblob) (1.11.0)


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
